In [ ]:
import json

file_in = open("dataset_articles.json")

papers = json.load(file_in)

def filter_papers(papers):
    res=[]
    for paper in papers:
        if paper["abstract"] and len(paper["abstract"]) > 150 and len(paper["keywords"]) not in [None,"."]:
            res.append(paper)

    return res

fi_papers = filter_papers(papers)

In [ ]:
from itertools import combinations
import re
pares=[]


def proc_keywords(keywords_s):
    keywords_s = keywords_s.lower().strip()
    keywords_s = re.sub(r"\\/","/",keywords_s)
    keywords=re.split(r"[,;/]",keywords_s)
    return [keywords.strip() for keyword in keywords]


#gerar pares
for paper1,paper2 in combinations(fi_papers,2):
    key1,key2 = proc_keywords(paper1["keywords"]), proc_keywords(paper2["keywords"])
    score = len(set(key1) & set(key2)) #intersecao, mas so é possivel em sets
    pares.append((paper1["abstract"],paper2["abstract"],score)) #é um tuplo

#pares[:10]
len(pares)

In [ ]:
from collections import Counter

Counter([score for _,_,score in pares])

In [ ]:
from sklearn.utils import resample
#balancear dados
majority_class = [(a1, a2, score) for a1, a2, score in pares if score ==0]
minority_class = [(a1, a2, score) for a1, a2, score in pares if score !=0]

undersampled_majority_class = resample(majority_class,
                                       replace=False #Dont duplicate samples
                                       n_samples = len(minortiy_class), #match minority
                                       random_state=42)

balanced_pairs = undersampled_majority_class + minority_class

Counter([score for _,_,score in balanced_pairs])

In [ ]:
def normalize_scores(score):
    if score == 1:
        return 0.5
    elif score ==2:
        return 0.7
    elif score >= 3:
        return 0.9
    
    elif score ==0:
        return 0
    
balanced_pairs = [(a1,a2,normalize_scores(s)) for a1, a2, s in balanced_pairs]

In [ ]:
from sklearn.model_selection import train_test_split

scores = [score for _, _, score in balanced_pairs]
train_data, test_data = train_test_split(
    balanced_pairs,
    test_size=2,
    random_state=42,
    stratify=scores
)


Counter([score for _,_,score in train_data])

Counter([score for _,_,score in test_data])

In [ ]:
#https://sbert.net/docs/sentence_transformer/training_overview.html
from datasets import Dataset
def create_dataset(pairs):
    res = {
        "abstracts1"=[],
        "abstracts2" = [],
        "score"=[]
    }

    for a1, a2, scores in pairs:
        res["abstracts1"].append(a1)
        res["abstracts2"].append(a2)
        res["scores"].append(scores)

    return res
test_dt = Dataset.from_dict(create_dataset(test_data))
train_dt = Dataset.from_dict(create_dataset(train_data))

In [ ]:

from sentence_transformers import SentenceTransformer
from sentence_transformers.losses import CoSENTLoss

# Load a model to train/finetune
model = SentenceTransformer("neuralmind/bert-base-portuguese-cased")

# Initialize the CoSENTLoss
# This loss requires pairs of text and a float similarity score as a label
loss = losses.CosineSimilarityLoss(model)



In [ ]:
from sentence_transformers import SentenceTransformerTrainer, SentenceTransformerTrainingArguments
from sentence_transformers.similarity_functions  import SimilarityFunction
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator
args = SentenceTransformerTrainingArguments(
    # Required parameter:
    output_dir="models/aula" ,
    report_to="none",
    # Optional training parameters:
    num_train_epochs=5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    fp16=True,  # Set to False if you get an error that your GPU can't run on FP16
    bf16=False,  # Set to True if you have a GPU that supports BF16
    # Optional tracking/debugging parameters:
    eval_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=2,
    load_best_model_at_end=True,
)


In [ ]:
#create evaluator
dev_evaluator = EmbeddingSimilarityEvaluator(
    test_dataset['abstract1'],  # Assuming these are the sentence pairs for evaluation
    test_dataset['abstract2'],
    test_dataset['score'],  # Assuming this contains the similarity scores
    main_similarity=SimilarityFunction.COSINE,
)

trainer = SentenceTransformerTrainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    loss=loss,
    evaluator=dev_evaluator,
)



In [ ]:
trainer.train

In [ ]:
model = SentenceTransformer("models/aula") #buscar o modelo


In [ ]:
abstracts = [paper["abstracts"] for paper in fi_papers]

titles = [paper["title"] for paper in fi_papers]
keywords = [paper["keywords"] for paper in fi_papers]



In [ ]:
from sentence_transformers import util
import torch
embeddings = model.encode(abstracts, convert_to_tensor=True)
query_embedding = model.encode(query_text, convert_to_tensor=True)
# Calculate the similarity between the query and the abstracts
cosine_scores = util.pytorch_cos_sim(query_embedding, embeddings)
retrieval_results = torch.topk(cosine_scores, k=15)

